In [ ]:
# Imports and WandB Init
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import fashion_mnist
import wandb
from wandb.keras import WandbCallback
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# WandB Setup
wandb.init(project="cnn-image-classifier", entity="cs24m023", name="VGG16-transfer-learning")

In [ ]:
# Data Preprocessing for VGG16
def preprocess_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

    # Resize to 224x224 and convert to 3 channels (RGB) for VGG16
    X_train = np.stack([np.repeat(tf.image.resize_with_pad(tf.convert_to_tensor(img[..., np.newaxis], dtype=tf.float32), 224, 224).numpy(), 3, axis=-1) for img in X_train])
    X_test = np.stack([np.repeat(tf.image.resize_with_pad(tf.convert_to_tensor(img[..., np.newaxis], dtype=tf.float32), 224, 224).numpy(), 3, axis=-1) for img in X_test])

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return X_train, y_train, X_test, y_test


In [ ]:
# Load Data
X_train, y_train, X_test, y_test = preprocess_fashion_mnist()


In [ ]:
# Define Transfer Learning Model using VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


In [ ]:
# Compile and Train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=32,
    callbacks=[WandbCallback()]
)


In [ ]:
# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Plot Accuracy vs Loss (Q8)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
